<a href="https://colab.research.google.com/github/aadhya2811/Fog-Detection/blob/main/selfdrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yolov5  # This installs the PyPI package (not Ultralytics official)

In [ ]:
!pip install tensorflow keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 116.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.19.0 which is incompatible.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have 

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thaslimvs/fog-detection-dataset")

print("Path to dataset files:", path)
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Step 1: Dataset Preparation
def load_fog_dataset(dataset_path):
    """Load images from fog and non-fog folders"""
    foggy_images = []
    clear_images = []

    # Paths to fog and non-fog folders
    foggy_path = os.path.join(dataset_path, 'fog images')
    clear_path = os.path.join(dataset_path, 'non-fog images')

    # Load foggy images
    for img_file in os.listdir(foggy_path)[:500]:  # Load first 500 for demo
        img_path = os.path.join(foggy_path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            foggy_images.append(img)

    # Load clear images
    for img_file in os.listdir(clear_path)[:500]:  # Load first 500 for demo
        img_path = os.path.join(clear_path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            clear_images.append(img)

    return np.array(foggy_images), np.array(clear_images)

# Load dataset
dataset_path = "/root/.cache/kagglehub/datasets/thaslimvs/fog-detection-dataset/versions/1/fog_dataset"
foggy_images, clear_images = load_fog_dataset(dataset_path)

print(f"Loaded {len(foggy_images)} foggy images")
print(f"Loaded {len(clear_images)} clear images")

## Step 2: Data Preprocessing
def preprocess_data(foggy_imgs, clear_imgs):
    # Normalize pixel values
    foggy_imgs = foggy_imgs.astype('float32') / 255.0
    clear_imgs = clear_imgs.astype('float32') / 255.0

    # Create labels (1 for foggy, 0 for clear)
    X = np.concatenate((foggy_imgs, clear_imgs), axis=0)
    y = np.concatenate((np.ones(len(foggy_imgs)), np.zeros(len(clear_imgs))))

    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = preprocess_data(foggy_images, clear_images)

## Step 3: Fog Detection Model
def build_fog_detection_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

fog_model = build_fog_detection_model()
history = fog_model.fit(X_train, y_train, epochs=10,
                        validation_data=(X_test, y_test))

## Step 4: Object Detection in Foggy Images
# Install YOLOv5 for object detection
!pip install -q torch torchvision
!git clone https://github.com/ultralytics/yolov5
!cd yolov5 && pip install -r requirements.txt

import torch
from yolov5 import YOLOv5

# Load pretrained YOLOv5 model
yolo_model = YOLOv5("yolov5s.pt")  # Small version for faster inference

## Step 5: Fog Reduction and Detection Pipeline
def enhance_foggy_image(image):
    """Improve visibility in foggy conditions"""
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)

    # Merge channels and convert back to RGB
    limg = cv2.merge((cl, a, b))
    enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)

    return enhanced

def process_image(img_path):
    """Complete processing pipeline for a single image"""
    # Load and preprocess image
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    original_img = img.copy()

    # Resize for fog detection model
    resized_img = cv2.resize(img, (256, 256))
    normalized_img = resized_img.astype('float32') / 255.0

    # Predict fog presence
    fog_prob = fog_model.predict(np.expand_dims(normalized_img, axis=0))[0][0]
    is_foggy = fog_prob > 0.5

    # Enhance if foggy
    if is_foggy:
        enhanced_img = enhance_foggy_image(img)
    else:
        enhanced_img = img

    # Detect objects
    results = yolo_model.predict(enhanced_img)

    # Visualization
    plt.figure(figsize=(20, 6))

    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(original_img)
    plt.title(f"Original\nFog Probability: {fog_prob:.2f}")
    plt.axis('off')

    # Enhanced image
    plt.subplot(1, 3, 2)
    plt.imshow(enhanced_img)
    plt.title("Enhanced" if is_foggy else "Original (No Enhancement Needed)")
    plt.axis('off')

    # Detection results
    plt.subplot(1, 3, 3)
    results.render()  # Draw bounding boxes
    plt.imshow(results.imgs[0])
    plt.title("Object Detection")
    plt.axis('off')

    plt.tight_layout()
    plt.show()

    # Print detection results
    print("\nDetected Objects:")
    for detection in results.pred[0]:
        class_id = int(detection[5])
        class_name = results.names[class_id]
        confidence = float(detection[4])
        print(f"- {class_name} (confidence: {confidence:.2f})")

    return {
        'original': original_img,
        'enhanced': enhanced_img,
        'detections': results,
        'is_foggy': is_foggy
    }

## Step 6: Example Usage
# Process sample foggy and clear images
sample_foggy = os.path.join(dataset_path, 'fog_images', os.listdir(os.path.join(dataset_path, 'fog_images'))[0])
sample_clear = os.path.join(dataset_path, 'non-fog_images', os.listdir(os.path.join(dataset_path, 'non-fog_images'))[0])

print("Processing foggy image:")
process_image(sample_foggy)

print("\nProcessing clear image:")
process_image(sample_clear)